# 교차검증
- `교차검증 cross validaion`

## 1. 표본 내 성능과 표본 외 성능
- 회귀분석 모형을 만드는 목적 중 하나는 종속 변수의 값을 아직 알지 못하고 따라서 학습에 사용하지 않은 표본에 대해 종속 변수의 값을 알아내고자 하는 것
- `표본 내 성능 in-sample testing` : 학습용 데이터 집합(training data set)의 종속 변수값을 얼마나 잘 예측하였는지 나타내는 성능
- `표본 외 성능 out-of-sample testing, 교차검증 cross validation` : 학습에 쓰이지 않은 데이터 집합의 종속 변수값을 얼마나 잘 예측하였는지 나타내는 성능
- **교차검증을 통해서 과최적화가 발생했는지 탐지할 수 있다.**

## 2. 과최적화
- `과최적화 overfitting` : 일반적으로 표본 내 성능과 표본 외 성능이 비슷하지만, 표본 내 성능은 좋으나 표본 외 성능이 상대적으로 많이 떨어지는 경우
    - 분석모형을 데이터에 과도하게 최적화 하는 것
    - 발생 : 독립변수의 수보다 모수의 수가 많은 경우, 독립변수간의 상관관계가 독립이 아닌 경우, 다항회귀를 과도하게 한 경우
    - 현상 : cross validaion 오차(오차가 커짐), 추정의 부정확함(모수 추정값이 크게 변함)
- 학습용 데이터에서는 종속 변수의 값을 잘 추정하지만, 새로운 데이터(테스트 용 데이터 등)가 주어지면 전혀 예측을 못하기 때문에 예측 목적으로 쓸모없는 모형이 된다.
    - 과최적화 발생하는 원인, 과최적화 방지를 위한 정규화(regularization)
    
## 3. 검증용 데이터 집합
- 교차검증을 위한 두 종류의 데이터 집합
    - 모형 추정 즉 학습을 위한 데이터 집합 (training data set)
    - 성능 검증을 위한 데이터 집합 (test data set)
- 두 데이터 모두 종속 변수값을 가지고 있어야 한다. 
    - 따라서 하나의 데이터 집합을 학습용과 검증용으로 나누어 사용한다.
    - 학습용 데이터로 회귀분석 모형을 만들고, 검증용 데이터로 성능을 계산한다.
- **학습/검증 데이터 분리 train-test split 방법**

## 4. statsmodels 패키지에서 교차검증
- 보스턴 집값 데이터로 학습, 검증 데이터 분리 후 회귀분석 모형 생성
- 데이터 분리를 직접 코드로 할 수 있다.

### 4-1. 데이터 임포트

In [1]:
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()
dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])
df = pd.concat([dfX, dfy], axis=1)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### 4-2. 학습, 검증용 데이터 분리

In [3]:
N = len(df)
ratio = 0.7
np.random.seed(0)

In [4]:
ratio * N

354.2

#### 0~504의 숫자에서 354개를 무작위로 선택

In [5]:
idx_train = np.random.choice(np.arange(N), np.int(ratio * N))
idx_train

array([172,  47, 117, 192, 323, 251, 195, 359,   9, 211, 277, 242, 292,
        87,  70, 472,  88, 396, 314, 193, 486,  39,  87, 174,  88, 337,
       165,  25, 333,  72, 265, 404, 115, 464, 243, 197, 335, 431, 448,
       338,  99, 472, 177, 243, 285, 147, 147, 398, 423, 288, 449, 265,
       185, 127,  32,  31, 202, 244, 151, 163, 459, 370, 183,  28, 290,
       128, 128, 420,  53, 389,  38, 488, 244, 273, 335, 388, 105,  42,
       442,  31, 376, 257, 321, 487, 425,  57, 291, 358, 119, 267, 430,
        82,  91, 384, 398,  99,  53, 396, 121, 426,  84, 203, 324, 262,
       452,  47, 127, 500, 131, 460, 356, 180, 488, 334, 143, 148, 227,
       442, 279, 207, 397, 373, 341,  48, 305,  69, 169, 163, 448,  95,
       197,  94, 256, 369, 178, 292, 418, 304, 349, 387,  98,  42, 461,
       368, 487, 405, 201, 383,   0, 394, 370,  43, 442, 383,  23, 187,
       130, 377,  98,  62, 419, 222, 123, 451,  82, 430, 227, 148, 209,
        50, 411, 270,  41,  58, 193,  36, 266,  86,  43, 360,  1

In [18]:
np.sort(idx_train)

array([  0,   3,   9,  11,  11,  13,  16,  19,  23,  24,  25,  25,  27,
        27,  28,  29,  31,  31,  32,  32,  32,  33,  35,  36,  36,  38,
        39,  40,  41,  41,  42,  42,  42,  43,  43,  43,  44,  47,  47,
        48,  50,  53,  53,  57,  58,  61,  62,  67,  67,  69,  70,  72,
        73,  77,  79,  80,  80,  82,  82,  83,  84,  86,  86,  87,  87,
        88,  88,  88,  91,  94,  94,  95,  95,  98,  98,  99,  99, 104,
       105, 109, 110, 111, 111, 115, 117, 117, 119, 121, 121, 123, 125,
       125, 127, 127, 128, 128, 128, 129, 129, 130, 131, 133, 136, 138,
       139, 143, 147, 147, 148, 148, 148, 149, 151, 151, 152, 152, 156,
       161, 163, 163, 165, 166, 169, 172, 174, 174, 174, 177, 178, 180,
       180, 182, 182, 183, 184, 184, 184, 185, 187, 189, 189, 192, 193,
       193, 195, 197, 197, 197, 197, 199, 201, 202, 203, 204, 207, 207,
       209, 211, 212, 214, 215, 216, 217, 218, 222, 223, 226, 227, 227,
       228, 232, 234, 237, 242, 243, 243, 244, 244, 248, 251, 25

In [26]:
repeat_num = []
repeat_list = []

for i in range(idx_train.shape[0]) : 
    if idx_train[i] not in repeat_num :
        count = 1
        for j in range(i+1, idx_train.shape[0]) :
            if idx_train[i] == idx_train[j] : 
                count += 1
                repeat_num.append(idx_train[i])
            else : 
                pass
        if count > 1 :
            repeat = (idx_train[i], count)
            repeat_list.append(repeat)

In [39]:
print(repeat_num)

[47, 117, 323, 251, 292, 87, 472, 88, 88, 396, 193, 486, 174, 174, 25, 265, 265, 464, 243, 197, 197, 197, 335, 448, 99, 147, 398, 423, 127, 32, 32, 31, 244, 151, 163, 459, 370, 290, 290, 128, 128, 53, 488, 488, 488, 42, 42, 442, 442, 487, 291, 267, 430, 82, 121, 180, 148, 148, 227, 207, 341, 95, 94, 256, 418, 349, 387, 98, 461, 368, 383, 394, 43, 43, 377, 377, 451, 270, 41, 36, 86, 360, 11, 258, 258, 307, 80, 182, 184, 184, 444, 125, 328, 152, 374, 287, 111, 129, 67, 259, 189, 27, 437]


In [40]:
print(repeat_list)

[(47, 2), (117, 2), (323, 2), (251, 2), (292, 2), (87, 2), (472, 2), (88, 3), (396, 2), (193, 2), (486, 2), (174, 3), (25, 2), (265, 3), (464, 2), (243, 2), (197, 4), (335, 2), (448, 2), (99, 2), (147, 2), (398, 2), (423, 2), (127, 2), (32, 3), (31, 2), (244, 2), (151, 2), (163, 2), (459, 2), (370, 2), (290, 3), (128, 3), (53, 2), (488, 4), (42, 3), (442, 3), (487, 2), (291, 2), (267, 2), (430, 2), (82, 2), (121, 2), (180, 2), (148, 3), (227, 2), (207, 2), (341, 2), (95, 2), (94, 2), (256, 2), (418, 2), (349, 2), (387, 2), (98, 2), (461, 2), (368, 2), (383, 2), (394, 2), (43, 3), (377, 3), (451, 2), (270, 2), (41, 2), (36, 2), (86, 2), (360, 2), (11, 2), (258, 3), (307, 2), (80, 2), (182, 2), (184, 3), (444, 2), (125, 2), (328, 2), (152, 2), (374, 2), (287, 2), (111, 2), (129, 2), (67, 2), (259, 2), (189, 2), (27, 2), (437, 2)]


#### N=504의 데이터에서 학습용 데이터의 idx를 제외

In [30]:
idx_test = list(set(np.arange(N)).difference(idx_train))
idx_test

[1,
 2,
 4,
 5,
 6,
 7,
 8,
 10,
 12,
 14,
 15,
 17,
 18,
 20,
 21,
 22,
 26,
 30,
 34,
 37,
 45,
 46,
 49,
 51,
 52,
 54,
 55,
 56,
 59,
 60,
 63,
 64,
 65,
 66,
 68,
 71,
 74,
 75,
 76,
 78,
 81,
 85,
 89,
 90,
 92,
 93,
 96,
 97,
 100,
 101,
 102,
 103,
 106,
 107,
 108,
 112,
 113,
 114,
 116,
 118,
 120,
 122,
 124,
 126,
 132,
 134,
 135,
 137,
 140,
 141,
 142,
 144,
 145,
 146,
 150,
 153,
 154,
 155,
 157,
 158,
 159,
 160,
 162,
 164,
 167,
 168,
 170,
 171,
 173,
 175,
 176,
 179,
 181,
 186,
 188,
 190,
 191,
 194,
 196,
 198,
 200,
 205,
 206,
 208,
 210,
 213,
 219,
 220,
 221,
 224,
 225,
 229,
 230,
 231,
 233,
 235,
 236,
 238,
 239,
 240,
 241,
 245,
 246,
 247,
 249,
 250,
 252,
 253,
 261,
 263,
 264,
 268,
 271,
 272,
 276,
 278,
 281,
 282,
 283,
 284,
 289,
 293,
 298,
 299,
 301,
 303,
 306,
 308,
 310,
 311,
 312,
 313,
 315,
 316,
 318,
 319,
 320,
 322,
 325,
 329,
 330,
 332,
 336,
 339,
 340,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 350,
 351,
 354,
 355,

In [31]:
len(idx_test)

255

In [32]:
N - len(idx_test)

251

In [33]:
df_train = df.iloc[idx_train]
df_test = df.iloc[idx_test]

In [34]:
len(df_train)

354

In [35]:
len(df_test)

255

### 4-3. 학습용 데이터로 회귀모형 만들기

In [36]:
model = sm.OLS.from_formula("MEDV~ " + "+".join(boston.feature_names), data=df_train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     81.31
Date:                Sat, 06 Aug 2022   Prob (F-statistic):           7.22e-96
Time:                        23:27:00   Log-Likelihood:                -1057.6
No. Observations:                 354   AIC:                             2143.
Df Residuals:                     340   BIC:                             2197.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.6105      6.807      5.966      0.0

### 4-4. 검증용 데이터로 성능 파악
- 결정계수 값으로 모형의 성능을 파악한다.

In [37]:
pred = result.predict(df_test)

# 잔차의 이동범위 : 잔차제곱합
rss = ((df_test.MEDV - pred) ** 2).sum()
# 종속변수의 이동범위
tss = ((df_test.MEDV - df_test.MEDV.mean()) ** 2).sum()
# 결정계수
rsquared = 1 - rss / tss
rsquared

0.6883734124987095

## 5. scikit-learn의 교차검증 기능
- 독립변수가 많은 빅데이터에서는 과최적화가 쉽게 발생한다.
- scikit-learn의 model_selection 서브 패키지는 교차검증을 위한 다양한 명령을 제공한다.
- `train_test_split()`
    - data : 독립변수 데이터 배열 또는 판다스 데이터 프레임
    - data2 : 종속변수 데이터 (data 인수에 종속변수 데이터가 같이 있으면 생략도 가능)
    - test_size : 검증용 데이터 개수. 1보다 작은 실수이면 비율
    - train_size : 학습용 데이터 개수. 1보다 작은 실수이면 비율. test_size와 train_size 둘 중 하나만 있어도 가능
    - random_seed : 난수 발생 시드

### 5-1. 보스턴 집값 데이터 학습, 검증 데이터 분리

In [42]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3, random_state=0)
df_train.shape, df_test.shape

((354, 14), (152, 14))

In [43]:
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


### 5-2. 이미 분리한 학습, 검증 데이터를 다시 분리

In [44]:
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(dfX, dfy,
                                                           test_size=0.3,
                                                           random_state=0)
dfX_train.shape, dfX_test.shape, dfy_train.shape, dfy_test.shape

((354, 13), (152, 13), (354, 1), (152, 1))

## 6. K-폴드 교차검증
- 데이터의 수가 적은 경우는 이 데이터의 일부인 검증 데이터도 적기 때문에 검증 성능의 신뢰도가 떨어지게 된다.
- 그렇다고 검증용 데이터의 수를 늘리면 학습 데이터의 수가 줄어들기 때문에 정상적인 학습이 되지 않는 문제가 발생한다.
- `K-폴드 교차검증 K-fold` : 데이터를 K개의 부분 집합으로 나눈 후 학습 데이터와 검증 데이터를 부분집합의 조합으로 분리하여 K개의 모형과 K의 교차검증 성능을 측정하는 방법 
    - 전체 데이터를 K개의 부분 집합으로 나누기 D_1, D_2, ..., D_k
    - D_1, D_2, ..., D_k-1을 학습 데이터로 회귀분석 모형을 만들고, D_k로 교차검증을 한다.
    - D_1, D_2, ..., D_k-2, D_k를 학습데이터로 회귀분석 모형을 만들고, D_k-1로 교차검증을 한다.
    - 이와 같은 방식으로 모든 부분 집합에 대해 학습, 검증 데이터를 분리하여 검증한다.

In [45]:
from sklearn.model_selection import KFold

In [46]:
scores = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
cv

KFold(n_splits=5, random_state=0, shuffle=True)

In [59]:
test_df = list(range(15))

for i, (idx_train, idx_test) in enumerate(cv.split(test_df)) : 
    print(i, (idx_train, idx_test))

0 (array([ 0,  2,  3,  4,  5,  7,  9, 10, 11, 12, 13, 14]), array([1, 6, 8]))
1 (array([ 0,  1,  2,  3,  5,  6,  7,  8, 10, 11, 12, 13]), array([ 4,  9, 14]))
2 (array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 11, 12, 14]), array([ 2, 10, 13]))
3 (array([ 0,  1,  2,  4,  5,  6,  8,  9, 10, 12, 13, 14]), array([ 3,  7, 11]))
4 (array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 13, 14]), array([ 0,  5, 12]))


In [64]:
for i, (idx_train, idx_test) in enumerate(cv.split(df)) : 
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]
    
    model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names),
                               data=df_train)
    # 모형 추정
    result = model.fit()
    
    # 모형 성능 검증
    pred = result.predict(df_test)
    rss = ((df_test.MEDV - pred) ** 2).sum()
    ## 종속변수의 이동범위 : 종속변수의 분산 : 데이터와 평균의 차이의 제곱의 합
    tss = ((df_test.MEDV - df_test.MEDV.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("k_{}, 학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(i, result.rsquared, rsquared))

k_0, 학습 R2 = 0.77301356, 검증 R2 = 0.58922238
k_1, 학습 R2 = 0.72917058, 검증 R2 = 0.77799144
k_2, 학습 R2 = 0.74897081, 검증 R2 = 0.66791979
k_3, 학습 R2 = 0.75658611, 검증 R2 = 0.66801630
k_4, 학습 R2 = 0.70497483, 검증 R2 = 0.83953317


## 7. 평가 점수
- scikit-learn의 metrics 서브패키지의 예측성능 평가 함수, 회귀분석에 유용한 대표적인 평가 기준.
    - `r2_score` : 결정계수
    - `mean_squared_error` : 평균 제곱 오차(mean squared error)
    - `median_absolute_error` : 절대 오차 중앙값(median absolute error)

In [66]:
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error

In [68]:
scores_r2 = np.zeros(5)
scores_mse = np.zeros(5)
scores_mae = np.zeros(5)

cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)) : 
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]
    
    model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names),
                               data=df_train)
    result = model.fit()
    pred = result.predict(df_test)
    
    rsquared = r2_score(df_test.MEDV, pred)
    mse = mean_squared_error(df_test.MEDV, pred)
    mae = median_absolute_error(df_test.MEDV, pred)
    
    scores_r2[i] = rsquared
    scores_mse[i] = mse
    scores_mae[i] = mae

In [69]:
scores_r2

array([0.58922238, 0.77799144, 0.66791979, 0.6680163 , 0.83953317])

In [70]:
scores_mse

array([33.44898   , 18.65881615, 21.23463289, 29.22251557, 16.57369039])

In [71]:
scores_mae

array([2.64327501, 2.77893757, 2.05618551, 2.41515299, 2.26594351])

## 8. 교차검증 반복
- `cross_val_score` : 교차검증을 간단하게 만들어주는 함수
    - model : 회귀분석 모형
    - x : 독립변수 데이터
    - y : 종속변수 데이터
    - scoring : 성능 검증에 사용할 함수의 이름
    - cv : 교차검증 생성기 객체 또는 숫자
        - None : KFold(3)
        - 숫자 k : KFold(k)
- 단 cross_val_score 명령은 scikit-learn에서 제공하는 모형만 사용가능하다.
- statsmodel의 OLS 객체를 사용하려면 scikit-learn의 RegressorMixin으로 wrapper class를 만들어 주어야 한다. 

In [72]:
from sklearn.base import BaseEstimator, RegressorMixin
import statsmodels.formula.api as smf
import statsmodels.api as sm

#### 래퍼 클래스 wrapper class

In [91]:
class StatsmodelsOLS(BaseEstimator, RegressorMixin) : 
    
    def __init__(self, formula) : 
        self.formula = formula
        self.model = None
        self.data = None
        self.result = None
        
    def fit(self, dfX, dfy) : 
        self.data = pd.concat([dfX, dfy], axis=1)
        self.model = smf.ols(self.formula, data=self.data)
        self.result = self.model.fit()
        
    def predict(self, new_data) : 
        return self.result.predict(new_data)

In [92]:
from sklearn.model_selection import cross_val_score

In [93]:
model = StatsmodelsOLS("MEDV ~ " + "+".join(boston.feature_names))
model

StatsmodelsOLS(formula='MEDV ~ '
                       'CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT')

In [96]:
cv = KFold(5, shuffle=True, random_state=0)
cross_val_score(model, dfX, dfy, scoring="r2", cv=cv)

array([0.58922238, 0.77799144, 0.66791979, 0.6680163 , 0.83953317])

## 모형 성능 증가
- 데이터 전처리
- 다중공선성 제거
- 분산분석을 통한 독립변수의 중요도 판단
- 독립변수, 종속변수의 비선형 변환
- 기저함수를 사용한 다항함수 적용
- 교차검증